In [2]:
# offic venv : py37Torch

In [3]:
!pip install BeautifulSoup4

In [4]:
import requests
from bs4 import BeautifulSoup

class crawler:

    def url_requests(self,address):
        try:
            req = requests.get(address,timeout=5)
        except:
            return "Timeout error.."
        source = req.text
        return BeautifulSoup(source, 'html.parser')

    def find_target_class_list(self,str_data,target_string):
        index = 0
        table_list = []
        while index > -1:
            index = str_data.find(target_string, index)
            if index > -1:
                table_info = str(str_data[index:str_data.find('>', index)])
                table_list.append(table_info.split("=")[1].split("\"")[1].split(" ")[0])
                index += 1
        return table_list

    def find_target_slicing(self,str_data,target_string):
        index = 0
        while index > -1:
            index = str_data.find(target_string, index)
            if index > -1:
                str_data = str_data[:index]
                index += 1
        return str_data

    def wikipedia_search(self,address):
        ################# 위키피디아 전용 ########################
        url_respond = self.url_requests(address)
        if url_respond == "Timeout error..":
            return "Timeout error.."
        else:
            contents = url_respond

            def section_info(request_instance,selector):
                info_txt = request_instance.select(selector)
                if(len(info_txt)>0):
                    return info_txt[0].text
                else:
                    return ""

            contents_info = contents.select("#mw-content-text > div.mw-parser-output")
            index_info = section_info(contents,"#toc")
            infobox_info = section_info(contents,"#mw-content-text > div.mw-parser-output > table.infobox.vcard")
            ref_info = section_info(contents,"#mw-content-text > div.mw-parser-output > div.reflist")

            filtered_info = contents_info[0].text.replace(index_info,"").replace(infobox_info,"").replace(ref_info,"")

            str_data = str(contents_info[0])
            table_list = self.find_target_class_list(str_data,'<table class=')

            table_result = list(map(lambda x: contents.select("."+str(x)), list(set(table_list))))


            for table_index in range(len(table_result)):
                for table_buffer in range(len(table_result[table_index])):
                    filtered_info = filtered_info.replace(str(table_result[table_index][table_buffer].text),"")

            ## Ref number 삭제
            for N_ref in list(map(lambda x: x.text, contents.select('.reference'))):
                filtered_info = filtered_info.replace(N_ref,"")

            ## Reference 이후 불필요한 부분 삭제
            filtered_info = self.find_target_slicing(filtered_info,"References[edit]")
            filtered_info = filtered_info.replace("[edit]","") ## edit 제거
            filtered_info = filtered_info.split("\n")

            strip_list = list(map(lambda x: x.strip(),filtered_info))

            sentence = ""
            for ingredient in strip_list:
                if(ingredient == ''):
                    pass
                else:
                    ingredient = ingredient.replace("\xa0"," ") ## 이따금 들어오는 공백 널문자는 공백으로 대체함.
                    ## 각 요소 마다 마지막엔 구두점으로 끝나게하자.(문장 구분을 통해 NLP성능 극대화)
                    if(ingredient[-1] == "."):
                        pass
                    else:
                        ingredient += "."
                    sentence += ingredient + " "

            return sentence[:-1]

cr = crawler()

address = "https://stories.starbucks.com/stories/the-starbucks-foundation/"

se_re = cr.url_requests(address)

se_re

!pip install openpyxl

!pip install selenium

# 구글 키워드로 검색하여 기업의 정보를 추출하는 기능 

In [5]:

import re
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from tqdm import tqdm

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

In [6]:
!pip install tqdm

In [7]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [8]:
from tqdm import tqdm

def google_search_result_tp_knowledge(input_word):
    baseUrl = 'https://www.google.com/search?q='

    #plusUrl = input('무엇을 검색할까요? :')
    plusUrl = input_word

    # url = baseUrl + quote_plus(plusUrl)
    url = baseUrl + plusUrl
    # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver', chrome_options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, features="html.parser")

    v = soup.select('.yuRUbf') # 링크추출
    # print(" get links : ", v)

    search_title_result = []
    search_linked_contents_result = []
    for i in v:
        #print(i.select_one('.LC20lb.DKV0Md').text)
        search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
        print(i.a.attrs['href'])
        search_linked_contents_result.append(i.a.attrs['href'])
        #print()

    print("search_linked_contents_result : ", search_linked_contents_result)
    
    # 링크로 접속해서 추가링크 모두 추출 --> 모듈로 만들어 적용
    search_linked_contents_result_ = []
    for get_lk_data in search_linked_contents_result:
            soup = BeautifulSoup(get_lk_data, features="html.parser")
            links = soup.find_all("a")
            for a in links:
                search_linked_contents_result_.append(a.attrs['href'])
                #print()
            
    # 추출한 모든 링크를 합쳐서 총 2단계 depth로 검색결과 확장
    search_linked_contents_result_sum = search_linked_contents_result + search_linked_contents_result_
    
    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, body내의 단어들만 추출한다.
    get_all_linked_web_data = []
    for linked_page in tqdm(search_linked_contents_result_sum):
        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    get_result_str = str(item_extract)
    result_cln = cleaning_data(get_result_str) # 결과값 청소
    result = list(set(result_cln)) # 중복제거

    return get_result_str, result

In [9]:
# run ##

#result = google_search_result(input_word)
result = google_search_result_tp_knowledge("Starbucks Coffee Company")

print(result)


/Users/kimkwangil/opt/anaconda3/envs/py37Torch/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]
/Users/kimkwangil/opt/anaconda3/envs/py37Torch/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "https://www.starbucks.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/kimkwangil/opt/anaconda3/envs/py37Torch/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "https://www.starbucks.co.kr/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/kimkwangil/opt/anaconda3/envs/py37Torch/lib/python3.7/site-packa

https://www.starbucks.com/
https://www.starbucks.co.kr/
https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4
https://en.wikipedia.org/wiki/Starbucks
https://www.britannica.com/topic/Starbucks
https://www.starbucksathome.com/story/about-us
https://play.google.com/store/apps/details?id=com.starbucks.mobilecard&hl=en_US&gl=US
https://www.facebook.com/Starbucks/
https://www.investopedia.com/articles/markets/101415/top-6-companies-owned-starbucks.asp
search_linked_contents_result :  ['https://www.starbucks.com/', 'https://www.starbucks.co.kr/', 'https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4', 'https://en.wikipedia.org/wiki/Starbucks', 'https://www.britannica.com/topic/Starbucks', 'https://www.starbucksathome.com/story/about-us', 'https://play.google.com/store/apps/details?id=com.starbucks.mobilecard&hl=en_US&gl=US', 'https://www.facebook.com/Starbucks/', 'https://www.investopedia.com/articles/markets/101415/top-6-companies-owned-starbucks.asp']


100%|██████████| 9/9 [00:21<00:00,  2.39s/it]


('                                                                                         LinkedIn with Background                                                                                                                                                                                         Education                    General         Dictionary         Economics         Corporate Finance         Roth IRA         Stocks         Mutual Funds         ETFs         401 k              Investing Trading         Investing Essentials         Fundamental Analysis         Portfolio Management         Trading Essentials         Technical Analysis         Risk Management                     Markets                    News         Company News         Markets News         Trading News         Political News         Trends             Popular Stocks         Apple  AAPL          Tesla  TSLA          Amazon  AMZN          AMD  AMD          Facebook  FB          Netflix  NFLX                   

# 결과물에 대한 Sentimental Analysis 진행한다. 검색어(기업명) 을 통한 검색정보를 통해서 기업의 평판을 분석해본다.
# 기업의 매출상황을 주식, 기업정보를 통해서 추출하여 비교해본다. 
# 최종 분석결과를 토대로 기업의 평판과 제품판매에 대한 상관관계를 분석해본다. 끝!

In [20]:
!pip install ipywidgets

  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
  Using cached jupyterlab_widgets-1.0.0-py3-none-any.whl (243 kB)
  Using cached widgetsnbextension-3.5.1-py2.py3-none-any.whl (2.2 MB)


In [1]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [12]:
classifier(result[1])

[{'label': 'POSITIVE', 'score': 0.9998362064361572},
 {'label': 'NEGATIVE', 'score': 0.7134793400764465},
 {'label': 'POSITIVE', 'score': 0.9933604001998901},
 {'label': 'POSITIVE', 'score': 0.9998142719268799},
 {'label': 'POSITIVE', 'score': 0.9882547855377197},
 {'label': 'NEGATIVE', 'score': 0.9249258637428284},
 {'label': 'POSITIVE', 'score': 0.9085069894790649},
 {'label': 'NEGATIVE', 'score': 0.9276038408279419},
 {'label': 'NEGATIVE', 'score': 0.9371795058250427},
 {'label': 'POSITIVE', 'score': 0.9960375428199768},
 {'label': 'NEGATIVE', 'score': 0.9870686531066895},
 {'label': 'POSITIVE', 'score': 0.9856825470924377},
 {'label': 'POSITIVE', 'score': 0.9547632932662964},
 {'label': 'POSITIVE', 'score': 0.9988128542900085},
 {'label': 'POSITIVE', 'score': 0.9989684224128723},
 {'label': 'POSITIVE', 'score': 0.9981051683425903},
 {'label': 'POSITIVE', 'score': 0.9762058258056641},
 {'label': 'POSITIVE', 'score': 0.9997578263282776},
 {'label': 'POSITIVE', 'score': 0.990890145301

In [18]:
positive_cnt = 0
negative_cnt = 0
for itm in classifier(result[1]):
    #print(itm)
    if itm['label'] == 'POSITIVE':
        positive_cnt += 1
    elif itm['label'] == 'NEGATIVE':
        negative_cnt += 1
    else:
        pass
    
print("positive_cnt", positive_cnt)
print("negative_cnt", negative_cnt)

positive_cnt 552
negative_cnt 120


In [21]:
RepRatio = round(positive_cnt / (negative_cnt + positive_cnt) * 100,1)
RepRatio

82.1

In [24]:
print("Positive analysis of corporate reputation results : {0} %".format(RepRatio))

Positive analysis of corporate reputation results : 82.1 %
